In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using StatsBase
include("utils.jl")
include("kmeans_moments_lattice.jl")
using .LatticeGeneration
using JLD2
using SDDP, HiGHS, Plots, XLSX, Mustache, PrettyTables

In [2]:
includet("goal_investing_sddp_model.jl")
using .GoalInvestingSDDP

In [99]:
scenario_grid = load_object("scenario_grid.jld2")
println(size(scenario_grid.probs))
println(size(scenario_grid.states))

(16,)
(15, 11, 2)


In [100]:
graph = SDDP.MarkovianGraph(scenario_grid.probs[1:end-1])

Root
 (0, 1)
Nodes
 (1, 1)
 (2, 1)
 (2, 2)
 (2, 3)
 (2, 4)
 (2, 5)
 (2, 6)
 (2, 7)
 (2, 8)
 (2, 9)
 (2, 10)
 (2, 11)
 (3, 1)
 (3, 2)
 (3, 3)
 (3, 4)
 (3, 5)
 (3, 6)
 (3, 7)
 (3, 8)
 (3, 9)
 (3, 10)
 (3, 11)
 (4, 1)
 (4, 2)
 (4, 3)
 (4, 4)
 (4, 5)
 (4, 6)
 (4, 7)
 (4, 8)
 (4, 9)
 (4, 10)
 (4, 11)
 (5, 1)
 (5, 2)
 (5, 3)
 (5, 4)
 (5, 5)
 (5, 6)
 (5, 7)
 (5, 8)
 (5, 9)
 (5, 10)
 (5, 11)
 (6, 1)
 (6, 2)
 (6, 3)
 (6, 4)
 (6, 5)
 (6, 6)
 (6, 7)
 (6, 8)
 (6, 9)
 (6, 10)
 (6, 11)
 (7, 1)
 (7, 2)
 (7, 3)
 (7, 4)
 (7, 5)
 (7, 6)
 (7, 7)
 (7, 8)
 (7, 9)
 (7, 10)
 (7, 11)
 (8, 1)
 (8, 2)
 (8, 3)
 (8, 4)
 (8, 5)
 (8, 6)
 (8, 7)
 (8, 8)
 (8, 9)
 (8, 10)
 (8, 11)
 (9, 1)
 (9, 2)
 (9, 3)
 (9, 4)
 (9, 5)
 (9, 6)
 (9, 7)
 (9, 8)
 (9, 9)
 (9, 10)
 (9, 11)
 (10, 1)
 (10, 2)
 (10, 3)
 (10, 4)
 (10, 5)
 (10, 6)
 (10, 7)
 (10, 8)
 (10, 9)
 (10, 10)
 (10, 11)
 (11, 1)
 (11, 2)
 (11, 3)
 (11, 4)
 (11, 5)
 (11, 6)
 (11, 7)
 (11, 8)
 (11, 9)
 (11, 10)
 (11, 11)
 (12, 1)
 (12, 2)
 (12, 3)
 (12, 4)
 (12, 5)
 (12, 

In [101]:
data_source = DataFrame(XLSX.readtable("./goal_data.xlsx", "goal_data_single_goal")) 
data = round.(data_source[1:16,:],digits=3)

Row,t,minimum_limit,acceptable_limit,desired_limit,inflows,priority,minimum_utility,acceptable_utility,desired_utility,above_desired_utility,cashflows
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,6.0,8.0,9.0,0.0,0.067,0.278,0.167,0.067,0.001,-9.0
2,2.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
3,3.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
4,4.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
5,5.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
6,6.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
7,7.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
8,8.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0
9,9.0,6.0,8.0,9.0,11.0,0.067,0.278,0.167,0.067,0.001,-2.0


In [103]:
model_data = GoalsData3(
    data.minimum_limit,
    data.acceptable_limit,
    data.desired_limit,
    data.inflows,
    data.minimum_utility,
    data.acceptable_utility,
    data.desired_utility,
    data.above_desired_utility,
    35.0,
    0.0029
    )

GoalsData3([6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 81.0], [8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 128.0], [9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 159.0], [0.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0], Float32[0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.278, 0.309], Float32[0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.167, 0.106], Float32[0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.067, 0.032], Float32[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001], 35.0, 0.0029f0)

In [104]:
model = asset_management_alm(model_data, scenario_grid)
SDDP.train(model; iteration_limit = 600, log_frequency = 50)


-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 166
  state variables : 2
  scenarios       : 4.17725e+15
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [14, 14]
  AffExpr in MOI.EqualTo{Float64}         : [4, 4]
  VariableRef in MOI.GreaterThan{Float64} : [12, 12]
  VariableRef in MOI.LessThan{Float64}    : [3, 4]
numerical stability report
  matrix range     [9e-01, 2e+00]
  objective range  [1e-03, 1e+00]
  bounds range     [1e+00, 8e+01]
  rhs range        [1e+01, 4e+01]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-----------------------------------------

In [121]:
result = simulate_policy(model)

Main.GoalInvestingSDDP.Optimistion_Result(-60.9469, -60.9562, 0.0799, [8.0 9.0 … 9.0 141.21861250802445; 8.0 9.0 … 8.0 106.41220377272728; … ; 8.0 9.0 … 8.0 86.23497829003196; 8.0 8.0 … 9.0 162.5104074770643], [26.921926295638798 26.921926295638798 … 26.921926295638798 26.921926295638798; 0.0 0.0 … 0.0 0.0;;; 31.666109816300466 36.88402713275082 … 31.666109816300466 28.02195480368391; 0.0 0.0 … 0.0 0.0;;; 34.82007473451528 40.62389862287719 … 35.15649164041029 36.96875710505143; 0.0 0.0 … 0.0 0.0;;; … ;;; 100.64632738030026 69.46457993133401 … 70.77267419563894 60.91423214411887; 0.0 0.0 … 0.0 92.21259895885237;;; 82.9601450680081 81.10611810901493 … 85.88441066235892 17.653619724121064; 32.21358035611711 0.0 … 0.0 131.59157162300562;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [122]:
result.lower_bound

-60.9469

In [107]:

function calculate_perc(data, perc = [0.05, 0.25, 0.5, 0.75, 0.95])
    n_scenarios, n_stages = size(data)
    
    result = zeros(n_stages, length(perc)) 

    for t in 1:n_stages
       result[t,:] = quantile(data[:,t], perc)
    end

    return DataFrame(round.(result,digits=2),string.(perc))
  
end

calculate_perc (generic function with 2 methods)

In [123]:
calculate_perc(result.consumption_dist)

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,8.0,8.0,8.0,8.0,8.0
2,8.0,8.0,9.0,9.0,9.0
3,8.0,8.0,9.0,9.0,9.0
4,8.0,8.0,9.0,9.0,9.0
5,8.0,8.27,9.0,9.0,9.0
6,8.0,8.39,9.0,9.0,9.0
7,8.0,8.84,9.0,9.0,9.0
8,8.0,9.0,9.0,9.0,9.0
9,8.0,9.0,9.0,9.0,9.0


In [124]:
# ret 3y
calculate_perc(result.assets_dist[1,:,:])


Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,26.92,26.92,26.92,26.92,26.92
2,27.27,28.18,31.67,35.89,37.79
3,28.54,34.08,36.97,42.2,50.04
4,32.38,38.68,42.99,49.8,62.1
5,36.0,42.89,49.38,58.32,74.69
6,38.64,47.55,56.0,66.75,87.04
7,41.89,52.51,63.3,77.19,103.45
8,46.47,58.78,71.53,88.38,119.47
9,50.7,65.15,79.65,99.25,139.87


In [125]:
wealth = dropdims(sum(result.assets_dist, dims=1), dims=1)
calculate_perc(wealth)

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,26.92,26.92,26.92,26.92,26.92
2,27.27,28.18,31.67,35.89,37.79
3,28.54,34.08,36.97,42.2,50.04
4,32.38,38.68,42.99,49.8,62.1
5,36.0,42.89,49.38,58.32,74.69
6,38.64,47.55,56.0,66.75,87.04
7,41.89,52.51,63.3,77.19,103.45
8,46.47,58.86,71.62,88.56,119.47
9,50.7,65.15,79.82,101.11,139.87


In [126]:
calculate_perc(result.assets_dist[1] ./ wealth)

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,1.0,1.0
2,0.71,0.75,0.85,0.96,0.99
3,0.54,0.64,0.73,0.79,0.94
4,0.43,0.54,0.63,0.7,0.83
5,0.36,0.46,0.55,0.63,0.75
6,0.31,0.4,0.48,0.57,0.7
7,0.26,0.35,0.43,0.51,0.64
8,0.23,0.3,0.38,0.46,0.58
9,0.19,0.27,0.34,0.41,0.53


In [127]:
total_consumption = sum(result.consumption_dist, dims=2)
calculate_perc(total_consumption)

Row,0.05,0.25,0.5,0.75,0.95
,Float64,Float64,Float64,Float64,Float64
1,217.8,264.48,292.95,335.07,473.51


total_consumption = 267.87, 293.17, 331.18
objective = - 61.05
2n porfolio - 74, 84, 90
initial consumption 8.26

In [114]:
report = print_report(result, model_data, "c:\\Users\\matsz\\programowanie\\Optymalizacja_portfela\\julia_msp\\reports\\tpl.html")

MethodError: MethodError: no method matching render(::String, ::Main.GoalInvestingSDDP.Optimistion_Result; TITLE::String, input_data::GoalsData3)

Closest candidates are:
  render(::AbstractString, ::Any; tags) got unsupported keyword arguments "TITLE", "input_data"
   @ Mustache C:\Users\matsz\.julia\packages\Mustache\d3gWb\src\render.jl:55
  render(!Matched::Mustache.MustacheTokens, ::Any) got unsupported keyword arguments "TITLE", "input_data"
   @ Mustache C:\Users\matsz\.julia\packages\Mustache\d3gWb\src\render.jl:30
  render(::AbstractString; kwargs...)
   @ Mustache C:\Users\matsz\.julia\packages\Mustache\d3gWb\src\render.jl:56


- simulate policy
- generate report summary
- save report